<a href="https://colab.research.google.com/github/Hanbin-git/dacon_new_drug/blob/main/start(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import zipfile

zip_path = '/content/drive/MyDrive/data.zip'
extract_path = '/content/project_data'  # 원하는 경로

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [3]:
import pandas as pd

path = '/content/project_data/'  # 압축 해제 경로
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
submission = pd.read_csv(path + 'sample_submission.csv')

print(train.shape, test.shape, submission.shape)


(1681, 3) (100, 2) (100, 2)


In [4]:
# RDKit 설치 (Colab에서 가능)
!pip uninstall -y rdkit-pypi
!pip install rdkit-pypi==2022.9.5 optuna xgboost



Found existing installation: rdkit-pypi 2022.9.5
Uninstalling rdkit-pypi-2022.9.5:
  Successfully uninstalled rdkit-pypi-2022.9.5
  Using cached rdkit_pypi-2022.9.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.9 kB)
Using cached rdkit_pypi-2022.9.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (29.4 MB)


In [5]:
# # ✅ RDKit + XGBoost + Optuna 전체 파이프라인
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import mean_squared_error
# from rdkit import Chem
# from rdkit.Chem import Descriptors, Lipinski, MACCSkeys, AllChem
# import xgboost as xgb
# import optuna
# import warnings
# warnings.filterwarnings("ignore")

# # ✅ 경로 설정
# path = '/content/project_data/'
# train = pd.read_csv(path + 'train.csv')
# test = pd.read_csv(path + 'test.csv')
# submission = pd.read_csv(path + 'sample_submission.csv')

# # ✅ 분자 특성 추출 함수
# def get_molecule_descriptors(smiles):
#     try:
#         mol = Chem.MolFromSmiles(smiles)
#         if mol is None: return [0] * 2232
#         basic = [
#             Descriptors.MolWt(mol), Descriptors.MolLogP(mol),
#             Descriptors.NumHAcceptors(mol), Descriptors.NumHDonors(mol),
#             Descriptors.TPSA(mol), Descriptors.NumRotatableBonds(mol),
#             Descriptors.NumAromaticRings(mol), Descriptors.NumHeteroatoms(mol),
#             Descriptors.FractionCSP3(mol), Descriptors.NumAliphaticRings(mol),
#             Lipinski.NumAromaticHeterocycles(mol), Lipinski.NumSaturatedHeterocycles(mol),
#             Lipinski.NumAliphaticHeterocycles(mol), Descriptors.HeavyAtomCount(mol),
#             Descriptors.RingCount(mol), Descriptors.NOCount(mol),
#             Descriptors.NHOHCount(mol), Descriptors.NumRadicalElectrons(mol)
#         ]
#         morgan = [int(b) for b in AllChem.GetMorganFingerprintAsBitVect(mol, 2, 2048).ToBitString()]
#         maccs = [int(b) for b in MACCSkeys.GenMACCSKeys(mol).ToBitString()]
#         return basic + morgan + maccs
#     except:
#         return [0] * 2232

# # ✅ 피처 생성
# train['features'] = train['Canonical_Smiles'].apply(get_molecule_descriptors)
# test['features'] = test['Canonical_Smiles'].apply(get_molecule_descriptors)
# X_train = np.array(train['features'].tolist())
# y_train = train['Inhibition'].values
# X_test = np.array(test['features'].tolist())

# # ✅ 정규화
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# # ✅ 평가 함수
# def normalized_rmse(y_true, y_pred):
#     return np.sqrt(mean_squared_error(y_true, y_pred)) / (np.max(y_true) - np.min(y_true))
# def pearson_correlation(y_true, y_pred):
#     return np.clip(np.corrcoef(y_true, y_pred)[0, 1], 0, 1)
# def competition_score(y_true, y_pred):
#     return 0.5 * (1 - min(normalized_rmse(y_true, y_pred), 1)) + 0.5 * pearson_correlation(y_true, y_pred)

# # ✅ Optuna 튜닝
# def objective(trial):
#     params = {
#         "n_estimators": trial.suggest_int("n_estimators", 300, 1000),
#         "max_depth": trial.suggest_int("max_depth", 4, 10),
#         "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
#         "subsample": trial.suggest_float("subsample", 0.6, 1.0),
#         "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
#         "reg_alpha": trial.suggest_float("reg_alpha", 0, 2),
#         "reg_lambda": trial.suggest_float("reg_lambda", 0, 2),
#         "random_state": 42
#     }
#     X_tr, X_val, y_tr, y_val = train_test_split(X_train_scaled, y_train, test_size=0.2, random_state=42)
#     model = xgb.XGBRegressor(**params)
#     model.fit(X_tr, y_tr, verbose=False)
#     y_val_pred = model.predict(X_val)
#     return -competition_score(y_val, y_val_pred)

# study = optuna.create_study(direction="minimize")
# study.optimize(objective, n_trials=30)

# # ✅ 최종 모델 학습 및 예측
# final_model = xgb.XGBRegressor(**study.best_params)
# final_model.fit(X_train_scaled, y_train)
# submission["Inhibition"] = final_model.predict(X_test_scaled)
# submission.to_csv("submission_optuna_xgb.csv", index=False)


In [6]:


# # ✅ 라이브러리 임포트
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import KFold
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import mean_squared_error
# from rdkit import Chem
# from rdkit.Chem import Descriptors, Lipinski, MACCSkeys, AllChem
# import xgboost as xgb
# import optuna
# import warnings
# warnings.filterwarnings("ignore")

# # ✅ 데이터 로딩
# path = '/content/project_data/'
# train = pd.read_csv(path + 'train.csv')
# test = pd.read_csv(path + 'test.csv')
# submission = pd.read_csv(path + 'sample_submission.csv')

# # ✅ 분자 특성 추출 함수
# def get_molecule_descriptors(smiles):
#     try:
#         mol = Chem.MolFromSmiles(smiles)
#         if mol is None: return [0] * 2232
#         basic = [
#             Descriptors.MolWt(mol), Descriptors.MolLogP(mol),
#             Descriptors.NumHAcceptors(mol), Descriptors.NumHDonors(mol),
#             Descriptors.TPSA(mol), Descriptors.NumRotatableBonds(mol),
#             Descriptors.NumAromaticRings(mol), Descriptors.NumHeteroatoms(mol),
#             Descriptors.FractionCSP3(mol), Descriptors.NumAliphaticRings(mol),
#             Lipinski.NumAromaticHeterocycles(mol), Lipinski.NumSaturatedHeterocycles(mol),
#             Lipinski.NumAliphaticHeterocycles(mol), Descriptors.HeavyAtomCount(mol),
#             Descriptors.RingCount(mol), Descriptors.NOCount(mol),
#             Descriptors.NHOHCount(mol), Descriptors.NumRadicalElectrons(mol)
#         ]
#         morgan = [int(b) for b in AllChem.GetMorganFingerprintAsBitVect(mol, 2, 2048).ToBitString()]
#         maccs = [int(b) for b in MACCSkeys.GenMACCSKeys(mol).ToBitString()]
#         return basic + morgan + maccs
#     except:
#         return [0] * 2232

# # ✅ 피처 생성
# train['features'] = train['Canonical_Smiles'].apply(get_molecule_descriptors)
# test['features'] = test['Canonical_Smiles'].apply(get_molecule_descriptors)
# X = np.array(train['features'].tolist())
# y = train['Inhibition'].values
# X_test = np.array(test['features'].tolist())

# # ✅ 정규화
# scaler = StandardScaler()
# X = scaler.fit_transform(X)
# X_test = scaler.transform(X_test)

# # ✅ 평가 함수
# def normalized_rmse(y_true, y_pred):
#     return np.sqrt(mean_squared_error(y_true, y_pred)) / (np.max(y_true) - np.min(y_true))
# def pearson_correlation(y_true, y_pred):
#     return np.clip(np.corrcoef(y_true, y_pred)[0, 1], 0, 1)
# def competition_score(y_true, y_pred):
#     return 0.5 * (1 - min(normalized_rmse(y_true, y_pred), 1)) + 0.5 * pearson_correlation(y_true, y_pred)

# # ✅ Optuna 튜닝 (1회만, 전체 데이터 기반)
# def objective(trial):
#     params = {
#         "n_estimators": trial.suggest_int("n_estimators", 300, 1000),
#         "max_depth": trial.suggest_int("max_depth", 4, 10),
#         "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
#         "subsample": trial.suggest_float("subsample", 0.6, 1.0),
#         "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
#         "reg_alpha": trial.suggest_float("reg_alpha", 0, 2),
#         "reg_lambda": trial.suggest_float("reg_lambda", 0, 2),
#         "random_state": 42
#     }
#     X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
#     model = xgb.XGBRegressor(**params)
#     model.fit(X_tr, y_tr, verbose=False)
#     y_val_pred = model.predict(X_val)
#     return -competition_score(y_val, y_val_pred)

# study = optuna.create_study(direction="minimize")
# study.optimize(objective, n_trials=30)
# best_params = study.best_params

# # ✅ KFold 앙상블 학습 + 예측
# kf = KFold(n_splits=5, shuffle=True, random_state=42)
# test_preds = np.zeros(X_test.shape[0])
# scores = []

# for fold, (tr_idx, val_idx) in enumerate(kf.split(X)):
#     print(f"📂 Fold {fold + 1}")
#     X_tr, X_val = X[tr_idx], X[val_idx]
#     y_tr, y_val = y[tr_idx], y[val_idx]

#     model = xgb.XGBRegressor(**best_params)
#     model.fit(X_tr, y_tr, verbose=False)
#     y_val_pred = model.predict(X_val)
#     score = competition_score(y_val, y_val_pred)
#     print(f"Fold {fold + 1} Score: {score:.4f}")
#     scores.append(score)

#     test_preds += model.predict(X_test) / 5  # 앙상블 평균

# # ✅ 후처리: 0~100 클리핑 + float32로 저장
# test_preds = np.clip(test_preds, 0, 100)
# submission['Inhibition'] = test_preds.astype(np.float32)
# submission.to_csv('submission_kfold_optuna.csv', index=False)

# print(f"\n✅ 평균 Score: {np.mean(scores):.4f}")
# print("📁 제출파일 저장 완료: submission_kfold_optuna.csv")


In [7]:
# import pandas as pd
# import numpy as np
# import os
# from sklearn.model_selection import KFold
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import mean_squared_error
# from sklearn.linear_model import Ridge
# from sklearn.ensemble import RandomForestRegressor
# from xgboost import XGBRegressor
# from lightgbm import LGBMRegressor
# from rdkit import Chem
# from rdkit.Chem import Descriptors, Lipinski, MACCSkeys, AllChem
# import warnings
# warnings.filterwarnings("ignore")


# # ✅ 데이터 로딩
# path = '/content/project_data/'
# train = pd.read_csv(path + 'train.csv')
# test = pd.read_csv(path + 'test.csv')
# submission = pd.read_csv(path + 'sample_submission.csv')


# # ✅ Feature 추출 함수
# def get_molecule_descriptors(smiles):
#     try:
#         mol = Chem.MolFromSmiles(smiles)
#         if mol is None:
#             return [0] * 2232

#         basic_descriptors = [
#             Descriptors.MolWt(mol),
#             Descriptors.MolLogP(mol),
#             Descriptors.NumHAcceptors(mol),
#             Descriptors.NumHDonors(mol),
#             Descriptors.TPSA(mol),
#             Descriptors.NumRotatableBonds(mol),
#             Descriptors.NumAromaticRings(mol),
#             Descriptors.NumHeteroatoms(mol),
#             Descriptors.FractionCSP3(mol),
#             Descriptors.NumAliphaticRings(mol),
#             Lipinski.NumAromaticHeterocycles(mol),
#             Lipinski.NumSaturatedHeterocycles(mol),
#             Lipinski.NumAliphaticHeterocycles(mol),
#             Descriptors.HeavyAtomCount(mol),
#             Descriptors.RingCount(mol),
#             Descriptors.NOCount(mol),
#             Descriptors.NHOHCount(mol),
#             Descriptors.NumRadicalElectrons(mol),
#         ]

#         morgan_fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)
#         morgan_features = [int(bit) for bit in morgan_fp.ToBitString()]
#         maccs_fp = MACCSkeys.GenMACCSKeys(mol)
#         maccs_features = [int(bit) for bit in maccs_fp.ToBitString()]

#         return basic_descriptors + morgan_features + maccs_features
#     except:
#         return [0] * 2232

# # ✅ Feature 생성
# train['features'] = train['Canonical_Smiles'].apply(get_molecule_descriptors)
# test['features'] = test['Canonical_Smiles'].apply(get_molecule_descriptors)
# X = np.array(train['features'].tolist())
# y = train['Inhibition'].values
# X_test = np.array(test['features'].tolist())

# # ✅ 스케일링
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)
# X_test_scaled = scaler.transform(X_test)

# # ✅ Metric
# def normalized_rmse(y_true, y_pred):
#     return np.sqrt(mean_squared_error(y_true, y_pred)) / (np.max(y_true) - np.min(y_true))

# def pearson_correlation(y_true, y_pred):
#     return np.clip(np.corrcoef(y_true, y_pred)[0,1], 0, 1)

# def competition_score(y_true, y_pred):
#     return 0.5 * (1 - normalized_rmse(y_true, y_pred)) + 0.5 * pearson_correlation(y_true, y_pred)

# # ✅ 모델 정의
# model = LGBMRegressor(
#     n_estimators=500, learning_rate=0.05,
#     max_depth=6, num_leaves=31,
#     subsample=0.8, colsample_bytree=0.8,
#     random_state=42
# )

# # ✅ 5-Fold 교차검증 + 예측
# kf = KFold(n_splits=5, shuffle=True, random_state=42)
# val_scores = []
# test_preds = []

# for fold, (train_idx, val_idx) in enumerate(kf.split(X_scaled)):
#     X_train_fold, X_val_fold = X_scaled[train_idx], X_scaled[val_idx]
#     y_train_fold, y_val_fold = y[train_idx], y[val_idx]

#     model.fit(X_train_fold, y_train_fold)
#     val_pred = model.predict(X_val_fold)
#     score = competition_score(y_val_fold, val_pred)
#     val_scores.append(score)
#     print(f"[Fold {fold+1}] 검증 점수: {score:.4f}")

#     test_pred = model.predict(X_test_scaled)
#     test_preds.append(test_pred)

# # ✅ 앙상블 (평균)
# final_test_pred = np.mean(test_preds, axis=0)

# # ✅ 후처리: 0~100 클리핑
# final_test_pred = np.clip(final_test_pred, 0, 100)

# # ✅ 제출 파일 저장
# submission['Inhibition'] = final_test_pred
# submission.to_csv("submission_kfold_lgbm.csv", index=False)
# print("✅ 제출 파일 저장 완료: submission_kfold_lgbm.csv")
# print(f"평균 검증 점수: {np.mean(val_scores):.4f}")


In [ ]:
!pip install numpy==1.23.5
import os
os.kill(os.getpid(), 9)  # 런타임 재시작 (필수)


In [1]:
!pip install rdkit-pypi catboost


In [2]:


# ✅ 라이브러리 로딩
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import Ridge
from sklearn.ensemble import StackingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler
from rdkit import Chem
from rdkit.Chem import AllChem, MACCSkeys, Descriptors, rdMolDescriptors

# ✅ 경로 설정 및 데이터 로딩
path = '/content/project_data/'
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
submission = pd.read_csv(path + 'sample_submission.csv')

# ✅ 1. SMILES 유효성 검사
def valid_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return mol is not None and mol.GetNumAtoms() > 1

train = train[train['Canonical_Smiles'].apply(valid_smiles)].reset_index(drop=True)

# ✅ 2. 중복 SMILES 평균처리
train = train.groupby("Canonical_Smiles").agg({
    "Inhibition": "mean"
}).reset_index()

# ✅ 3. Canonical SMILES 재정의
def canonicalize(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return Chem.MolToSmiles(mol, canonical=True) if mol else smiles

train["Canonical_Smiles"] = train["Canonical_Smiles"].apply(canonicalize)
test["Canonical_Smiles"] = test["Canonical_Smiles"].apply(canonicalize)

# ✅ 4. 확장된 Descriptors 함수 정의
def get_expanded_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return [0]*5
    return [
        Descriptors.MolMR(mol),
        rdMolDescriptors.CalcExactMolWt(mol),  # MolVol → 분자량으로 대체
        Descriptors.BalabanJ(mol),
        Descriptors.Chi0(mol),
        Descriptors.Kappa1(mol),
    ]

# ✅ 5. Feature 생성 함수 정의
def get_features(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return np.zeros(512)

    morgan = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=256)
    maccs = MACCSkeys.GenMACCSKeys(mol)
    extra = get_expanded_descriptors(smiles)

    return np.concatenate([
        np.array(morgan),
        np.array(maccs),
        np.array(extra)
    ])

# ✅ 전체 feature 변환
X = np.array([get_features(s) for s in train['Canonical_Smiles']])
X_test = np.array([get_features(s) for s in test['Canonical_Smiles']])
y = train['Inhibition'].values

# ✅ 표준화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

# ✅ 모델 정의
base_models = [
    ('xgb', XGBRegressor(n_estimators=300, learning_rate=0.1, random_state=42)),
    ('lgb', LGBMRegressor(n_estimators=300, learning_rate=0.1, random_state=42)),
    ('ridge', Ridge(alpha=1.0))
]
meta_model = Ridge()

stack_model = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5,
    n_jobs=-1
)

# ✅ KFold 학습 + 추론
kf = KFold(n_splits=5, shuffle=True, random_state=42)
preds = np.zeros(len(X_test_scaled))
for fold, (tr_idx, val_idx) in enumerate(kf.split(X_scaled, y)):
    X_train, X_val = X_scaled[tr_idx], X_scaled[val_idx]
    y_train, y_val = y[tr_idx], y[val_idx]

    stack_model.fit(X_train, y_train)
    val_pred = stack_model.predict(X_val)
    print(f'Fold {fold+1} MAE:', mean_absolute_error(y_val, val_pred))

    preds += stack_model.predict(X_test_scaled) / kf.n_splits

# ✅ 결과 저장
submission['Inhibition'] = preds
submission.to_csv('/content/submission_stack.csv', index=False)


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Fold 1 MAE: 20.50232411022973


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Fold 2 MAE: 20.056287300003262


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Fold 3 MAE: 19.282389806215498


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Fold 4 MAE: 21.050464016475864
Fold 5 MAE: 21.122982024070637


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [ ]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, MACCSkeys, AllChem
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

# ✅ 데이터 경로
path = '/content/project_data/'
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
submission = pd.read_csv(path + 'sample_submission.csv')

# ✅ 유효한 SMILES 필터링
def valid_smiles(smi):
    mol = Chem.MolFromSmiles(smi)
    return mol is not None and mol.GetNumAtoms() > 1

train = train[train['Canonical_Smiles'].apply(valid_smiles)].reset_index(drop=True)

# ✅ Canonicalization
def canonicalize(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return Chem.MolToSmiles(mol, canonical=True) if mol else smiles

train['Canonical_Smiles'] = train['Canonical_Smiles'].apply(canonicalize)
test['Canonical_Smiles'] = test['Canonical_Smiles'].apply(canonicalize)

# ✅ Feature 생성
def get_features(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return np.zeros(512 + 167 + 5)

    morgan = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=512)
    morgan_np = np.array(morgan)

    maccs = MACCSkeys.GenMACCSKeys(mol)
    maccs_np = np.array(maccs)[1:]

    others = np.array([
        Descriptors.MolMR(mol),
        Descriptors.BalabanJ(mol),
        Descriptors.Chi0(mol),
        Descriptors.Chi1(mol),
        Descriptors.Kappa1(mol)
    ])

    return np.concatenate([morgan_np, maccs_np, others])

X = np.array([get_features(smi) for smi in train['Canonical_Smiles']])
X_test = np.array([get_features(smi) for smi in test['Canonical_Smiles']])
y = train['Inhibition'].values

scaler = StandardScaler()
X = scaler.fit_transform(X)
X_test = scaler.transform(X_test)

# ✅ KFold 설정
kf = KFold(n_splits=5, shuffle=True, random_state=42)

def run_stacking(meta_model, file_name):
    base_models = [
        ('xgb', XGBRegressor(n_estimators=300, random_state=42)),
        ('lgbm', LGBMRegressor(n_estimators=300, random_state=42)),
        ('cat', CatBoostRegressor(n_estimators=300, verbose=0, random_state=42))
    ]

    stack = StackingRegressor(
        estimators=base_models,
        final_estimator=meta_model,
        passthrough=True,
        n_jobs=-1
    )

    preds = np.zeros(len(X_test))
    for train_idx, val_idx in kf.split(X):
        X_tr, X_val = X[train_idx], X[val_idx]
        y_tr, y_val = y[train_idx], y[val_idx]
        stack.fit(X_tr, y_tr)
        preds += stack.predict(X_test) / kf.n_splits

    submission['Inhibition'] = preds
    submission.to_csv(file_name, index=False)

# ✅ 전략 2: Meta → LinearRegression
run_stacking(LinearRegression(), 'submission_stack_linear.csv')

# ✅ 전략 3: Meta → LGBM (CatBoost 포함)
run_stacking(LGBMRegressor(n_estimators=100, random_state=42), 'submission_stack_lgbm.csv')


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.016211 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3975
[LightGBM] [Info] Number of data points in the train set: 1344, number of used features: 653
[LightGBM] [Info] Start training from score 33.391242


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.015918 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3969
[LightGBM] [Info] Number of data points in the train set: 1345, number of used features: 651
[LightGBM] [Info] Start training from score 33.637152


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.019082 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3972
[LightGBM] [Info] Number of data points in the train set: 1345, number of used features: 652
[LightGBM] [Info] Start training from score 33.054065


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
